In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class CompareModel(nn.Module):
    def __init__(self, emb_dim, dense_size, dropout, output_size):
        super().__init__()
        
        # Define the layers
        self.conv1 = nn.Conv1d(emb_dim, 32, kernel_size=5, padding=2)  # Assuming input channels=1
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.maxpool = nn.MaxPool1d(kernel_size=3)
        self.global_avgpool = nn.AdaptiveAvgPool1d(1)  # Global average pooling
        self.dense1 = nn.Linear(64, dense_size)
        self.dropout = nn.Dropout(dropout)
        self.dense2 = nn.Linear(dense_size, output_size)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Input shape: (batch_size, seq_len, geom_vector_len)
        # Convolutional layers
        x = x.permute(0, 2, 1)  # Permute to (batch_size, channels, seq_len)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.global_avgpool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)  # Reshape to (batch_size, num_features)
        
        # Fully connected layers
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.dense2(x)
        
        # No need to add softmax (already included in CrossEntropyLossFunction), otherwise it will be double softmax and converge slower

        return x

In [71]:
import numpy as np

train_loaded = np.load("archaeology_train_v8.npz", allow_pickle=True)
train_geoms = train_loaded['geoms']
train_labels = train_loaded['feature_type']

dataset_size = 1000
train_geoms = train_geoms[:1000]
train_labels = train_labels[:1000]

# Normalize
import geom_scaler

gs = geom_scaler.scale(train_geoms)
train_geoms = geom_scaler.transform(train_geoms, gs)

In [72]:
zipped = zip(train_geoms, train_labels)
train_input_sorted = {}
train_labels_sorted = {}

for geom, label in sorted(zipped, key=lambda x: len(x[0]), reverse=True):
    seq_len = geom.shape[0]
    if seq_len in train_input_sorted:
        train_input_sorted[seq_len].append(geom)
        train_labels_sorted[seq_len].append(label)
    else:
        train_input_sorted[seq_len] = [geom]
        train_labels_sorted[seq_len] = [label]

In [77]:
# Create training data
sequence_length = 10
geom_vector_len = 5  # Assuming geom_vector_len is known
dense_size = 64  # Size of the dense layer
dropout = 0.5  # Dropout rate
num_classes = 10  # Number of output classes
batch_size = 32
dataset_size = batch_size*50

# Define the model, loss function, and optimizer
conv_model = CompareModel(emb_dim=geom_vector_len, dense_size=dense_size, dropout=dropout, output_size=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(conv_model.parameters(), lr=0.001)

# Training process
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    for seq_len in train_input_sorted:
        inputs = torch.tensor(train_input_sorted[seq_len], dtype=torch.float32)
        labels = torch.tensor(train_labels_sorted[seq_len], dtype=torch.long)
        dataset = TensorDataset(inputs, labels)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        for batch_x, batch_y in loader:
            optimizer.zero_grad()
            output = conv_model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            # Print statistics
            running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/train_geoms.shape[0]}")

/tmp/ipykernel_440396/2313122069.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  inputs = torch.tensor(train_input_sorted[seq_len], dtype=torch.float32)


Epoch 1, Loss: 0.24168250457942486
Epoch 2, Loss: 0.21854528646357357
Epoch 3, Loss: 0.2042471772953868
Epoch 4, Loss: 0.2068303344398737
Epoch 5, Loss: 0.19663107278570532
Epoch 6, Loss: 0.19005830293893813
Epoch 7, Loss: 0.18144561527576297
Epoch 8, Loss: 0.18172121848911046
Epoch 9, Loss: 0.17903948907554149
Epoch 10, Loss: 0.18372297078371047
Epoch 11, Loss: 0.17875683285295962
Epoch 12, Loss: 0.1758677265048027
Epoch 13, Loss: 0.18768857842683792
Epoch 14, Loss: 0.1729503243714571
Epoch 15, Loss: 0.17895330633223056
Epoch 16, Loss: 0.16413020990043878
Epoch 17, Loss: 0.17462833605706693
Epoch 18, Loss: 0.17081272372603418
Epoch 19, Loss: 0.17014122991263866
Epoch 20, Loss: 0.1620531058870256
Epoch 21, Loss: 0.16939565777778626
Epoch 22, Loss: 0.16588488138467072
Epoch 23, Loss: 0.16204914071410895
Epoch 24, Loss: 0.16486501970514655
Epoch 25, Loss: 0.1608663150370121
Epoch 26, Loss: 0.15873472491651774
Epoch 27, Loss: 0.15681633777357637
Epoch 28, Loss: 0.15612166936695576
Epoch 2